In [93]:
import torch
TORCH = torch.__version__.split('+')[0]
CUDA = 'cu' + torch.version.cuda.replace('.', '')

# 2. Install torch-scatter, torch-sparse, and finally, torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.9.0+cu126.html


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
import networkx as nx
from torch_geometric.utils import to_networkx
import pandas as pd
import numpy as np

# Load the Cora citation network dataset using PyTorch Geometric's Planetoid loader.
# The 'Cora' dataset consists of machine learning papers classified into seven categories.
dataset = Planetoid(root='./data/Cora', name='Cora')
# 'data' is a torch_geometric.data.Data object containing the graph structure and features.
data = dataset[0]

# Convert the PyG Data object to a NetworkX graph (G) for using built-in NetworkX
# graph analysis functions, specifically making it undirected for centrality calculations.
G = to_networkx(data, to_undirected=True)

# Print basic statistics about the Cora dataset
print(f"Nodes: {data.num_nodes}") # Total number of nodes (papers)
print(f"Features: {data.num_node_features}") # Dimension of the feature vector (Bag-of-Words) for each node
print(f"Classes: {dataset.num_classes}") # Number of distinct classes (categories)

# 'edge_index' stores the graph connectivity in COO format (2 x num_edges tensor).
edge_index = data.edge_index


# Set a limit for printing edges (though no edges are printed in the final output)
num_edges_to_print = min(5, edge_index.shape[1])

# Print the total number of edges (directed pairs)
print(f"\nTotal number of directed edges: {data.num_edges}")

Nodes: 2708
Features: 1433
Classes: 7

Total number of directed edges: 10556


In [ ]:
# Calculate Degree Centrality: Measures the number of connections (neighbors) a node has.
# For an undirected graph, this indicates how many other papers a given paper cites or is cited by.
degree_centrality = nx.degree_centrality(G)

# Calculate Closeness Centrality: Measures how quickly a node can reach all other nodes in the network.
# A higher score means the node is more central in terms of reachability and information flow speed.
closeness_centrality = nx.closeness_centrality(G)

# Calculate Betweenness Centrality: Measures how often a node lies on the shortest path between two other nodes.
# A high score indicates the node acts as an important 'bridge' or intermediary in the network.
betweenness_centrality = nx.betweenness_centrality(G)

# Consolidate the calculated centrality scores into a single Pandas DataFrame.
node_properties_df = pd.DataFrame({
    # Get all node IDs from the NetworkX graph
    'node_id': list(G.nodes()),
    # Map the centrality dictionary results back to the node IDs
    'degree_centrality': [degree_centrality[n] for n in G.nodes()],
    'closeness_centrality': [closeness_centrality[n] for n in G.nodes()],
    'betweenness_centrality': [betweenness_centrality[n] for n in G.nodes()]
}).set_index('node_id') # Set the 'node_id' column as the DataFrame index

# Print the first few rows of the DataFrame to inspect the calculated properties.
print("Sample Node Properties:")
print(node_properties_df.head())

Sample Node Properties:
         degree_centrality  closeness_centrality  betweenness_centrality
node_id                                                                 
0                 0.001108              0.144255            9.766154e-07
1                 0.001108              0.151453            1.080477e-03
2                 0.001847              0.179168            4.050816e-03
3                 0.000369              0.000369            0.000000e+00
4                 0.001847              0.153266            5.511762e-04


In [ ]:

for col in node_properties_df.columns:
    node_properties_df[f'{col}_rank'] = node_properties_df[col].rank(pct=True) * 100

print("\nSample Node Properties with Ranks:")
print(node_properties_df[['degree_centrality', 'degree_centrality_rank']].head())


Sample Node Properties with Ranks:
         degree_centrality  degree_centrality_rank
node_id                                           
0                 0.001108               49.667651
1                 0.001108               49.667651
2                 0.001847               79.431315
3                 0.000369                8.973412
4                 0.001847               79.431315


In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

# Function to align (reduce the dimensionality of) feature vectors using SVD.
# feature_matrix: The input feature matrix (Nodes x Features).
# target_dim: The desired number of dimensions for the output features.
def align_features_with_svd(feature_matrix: np.ndarray, target_dim: int) -> np.ndarray:
    # Check if the input feature matrix is empty.
    if feature_matrix.size == 0:
        print("Error: Feature matrix is empty. SVD requires node features.")
        return np.array([])
        
    # Step 1: Normalize the feature vectors (rows) using L2 norm. 
    # This ensures all input vectors have unit length before SVD.
    normalized_features = normalize(feature_matrix, axis=1, norm='l2')

    # Step 2: Perform Singular Value Decomposition (SVD).
    # Decomposes normalized_features into U, S, and V_transpose.
    # U: Left singular vectors (orthogonal basis for the column space).
    # S: Singular values (magnitudes of the basis vectors).
    # Vt: Right singular vectors (transposed, orthogonal basis for the row space).
    U, S, Vt = np.linalg.svd(normalized_features, full_matrices=False)
    
    # Step 3: Determine the rank (r) for truncation.
    # r is the minimum of the desired target dimension and the number of columns in U.
    r = min(target_dim, U.shape[1])
    
    # Truncate U: Select the first 'r' columns corresponding to the largest singular values.
    U_r = U[:, :r]
    
    # Truncate S: Select the first 'r' singular values and form a diagonal matrix.
    S_r = np.diag(S[:r])
    
    # Step 4: Compute the aligned features using the product of U_r and S_r.
    # This represents the dimensionality-reduced feature set.
    aligned_features = U_r @ S_r

    # Print the shapes for verification and debugging.
    print(f"Original shape: {feature_matrix.shape}")
    print(f"Aligned shape: {aligned_features.shape}")

    return aligned_features

In [ ]:

Cora_features = np.random.rand(2708, 1433).astype(np.float32)

# Pubmed: 19717 nodes, 500 features
Pubmed_features = np.random.rand(19717, 500).astype(np.float32)

TARGET_ALIGN_DIM = 128

print("--- Aligning Cora Features ---")
aligned_cora = align_features_with_svd(Cora_features, TARGET_ALIGN_DIM)

print("\n--- Aligning Pubmed Features ---")
aligned_pubmed = align_features_with_svd(Pubmed_features, TARGET_ALIGN_DIM)

# Output of above print statements
##--- Aligning Cora Features ---
## Original shape: (2708, 1433)
## Aligned shape: (2708, 128)

##--- Aligning Pubmed Features ---
## Original shape: (19717, 500)
## Aligned shape: (19717, 128)

In [ ]:
import pandas as pd
import networkx as nx

# 1. Load Node Features and Labels (cora.content)
# cora.content contains the node ID, a binary Bag-of-Words (BoW) feature vector, and the class label.
# It is read as a TSV (Tab Separated Values) file without a header.
cora_content = pd.read_csv("/content/cora/cora.content", sep="\t", header=None)

# Assign custom column names to make the data understandable:
# - "id": The paper's unique identifier (node ID).
# - "w{i}": The 1433 binary feature words.
# - "class": The final classification label (e.g., Neural Networks, Genetic Algorithms).
cora_content.columns = ["id"] + [f"w{i}" for i in range(1, cora_content.shape[1]-1)] + ["class"]

# 2. Load Edge Data (cora.cites)
# cora.cites defines the citation relationships (edges).
edges = pd.read_csv("/content/cora/cora.cites", sep="\t", header=None, names=["source", "target"])

# 3. Create the NetworkX Graph
# Build a NetworkX graph G from the list of citation edges.
# The graph is explicitly created as a Directed Graph (DiGraph) since citations are directional.
G = nx.from_pandas_edgelist(edges, source="source", target="target", create_using=nx.DiGraph())

# 4. Define Text Reconstruction Function
# The original features are a binary vector (1=word present, 0=word absent).
# This function reconstructs a pseudo-text string by listing all words (column names starting with 'w') that have a value of 1.
def bow_to_text(row):
    words = [col for col in row.index if col.startswith("w") and row[col] == 1]
    return " ".join(words)

# 5. Map Node IDs to Reconstructed Text
# Create a dictionary mapping each paper's unique ID to its reconstructed pseudo-text string.
node_text = {
    row["id"]: bow_to_text(row)  
    for _, row in cora_content.iterrows()
}

# 6. Utility Function for Text Retrieval
# Provides safe access to the reconstructed text for any given node ID.
def get_original_text(node_id):
    if node_id not in node_text:
      return "No text found for this node."
    return node_text[node_id]

In [98]:
common_nodes = list(set(G.nodes()).intersection(node_text.keys()))
sample_node_id = common_nodes[0]

In [ ]:
import numpy as np

# Function to retrieve the text content of a sample of connected neighbor nodes
# It's crucial for generating the Text-Attributed Node Sample (TANS) prompt, 
# as neighbor text provides contextual information for classification.
def get_neighbor_texts(graph, node_id, num_neighbors=5):
    
    # 1. Check if the target node exists in the graph.
    if node_id not in graph:
        return "Node not found in graph."

    # 2. Get the list of all connected nodes (neighbors).
    neighbors = list(graph.neighbors(node_id))

    # 3. Handle case where the node has no connections.
    if len(neighbors) == 0:
        return "No connected nodes found."

    # 4. Randomly select a fixed number of neighbors (up to num_neighbors).
    selected = np.random.choice(
        neighbors,
        min(num_neighbors, len(neighbors)), # Ensure we don't try to sample more neighbors than exist
        replace=False # Do not pick the same neighbor twice
    )

    # 5. Format and return the neighbor texts.
    # It constructs a string where each line contains the neighbor ID and a truncated version of its text.
    return "\n".join(
        [
            # 'get_original_text(n)' is assumed to be a previously defined function
            f"{n}: {get_original_text(n)[:200]}..." 
            for n in selected
        ]
    )

# --- Demonstration and Testing ---

# Select a test node (using the ID of the first paper in the loaded cora.content file)
# 'cora_content' and 'get_original_text' are assumed to be defined in previous cells.
test_node = cora_content.iloc[0]["id"]

# Print the text content of the target node
print("--- Original Text ---")
print(get_original_text(test_node))

# Print the text content of the sample node's neighbors
print("\n--- Neighbor Texts ---")
# 'G' is the NetworkX graph loaded previously
print(get_neighbor_texts(G, test_node, num_neighbors=5))

--- Original Text ---
w119 w126 w177 w253 w352 w457 w508 w522 w620 w649 w699 w703 w735 w846 w903 w1206 w1210 w1237 w1353 w1427

--- Neighbor Texts ---
686532: w133 w174 w212 w329 w330 w336 w435 w522 w565 w704 w726 w730 w798 w1171 w1209 w1212 w1258 w1302 w1329 w1340 w1424 w1426...
31349: w457 w649 w903 w1210 w1274...
1129442: w133 w136 w232 w238 w251 w265 w331 w469 w699 w875 w903 w1020 w1098 w1136 w1274 w1349 w1353 w1360...


In [ ]:
# Function to assemble the Text-Attributed Node Sample (TANS) prompt for the LLM.
# It packages textual and structural information into a structured request.
def generate_tans_prompt(graph, node_id, properties_df, classes):
    
    # Retrieve pre-calculated centrality and rank values for the target node.
    degree = properties_df.loc[node_id, 'degree_centrality']
    rank_degree = properties_df.loc[node_id, 'degree_centrality_rank']

    # Get the text for the target node (assumes 'get_original_text' is defined).
    original_text = get_original_text(node_id)

    # Get the text for a sample of the neighbor nodes (assumes 'get_neighbor_texts' is defined).
    neighbor_texts = get_neighbor_texts(graph, node_id, num_neighbors=5)

    # Construct the final prompt using an f-string template.
    prompt = f"""
Given a node from a citation network graph, where the node type is paper.
The original node description is: "{original_text}".

The following are the textual information of 5 connected nodes. The descriptions are:
{neighbor_texts}

Node Properties:
# Include key structural metrics and their ranks.
- Degree Centrality value: {degree:.4f}, ranked as {rank_degree:.2f}% among all nodes.
- Closeness Centrality value: {properties_df.loc[node_id, 'closeness_centrality']:.4f}.
- Betweenness Centrality value: {properties_df.loc[node_id, 'betweenness_centrality']:.4f}.

# Final instruction: specify the task, the class list, the required reasoning, and the length constraint.
Output the potential class of the node among the following classes: {classes}.
Provide reasons for your assessment. Your answer should be less than 200 words.
"""
    # Remove leading/trailing whitespace and return the final prompt string.
    return prompt.strip()

In [ ]:

common_nodes = list(
    set(G.nodes()).intersection(node_text.keys()).intersection(node_properties_df.index)
)

# Pick a sample node
sample_node_id = common_nodes[0]  # safe


In [102]:
print("Sample node ID:", sample_node_id)
print("In G:", sample_node_id in G)
print("In node_text:", sample_node_id in node_text)
print("In node_properties_df:", sample_node_id in node_properties_df.index)

Sample node ID: 128
In G: True
In node_text: True
In node_properties_df: True


In [ ]:

classes_cora = ["Neural Networks", "Probabilistic Methods", "Genetic Algorithms", "Theory", "Case Based", "Reinforcement Learning", "Rule Learning"]


common_nodes = list(
    set(G.nodes()).intersection(node_text.keys()).intersection(node_properties_df.index)
)

sample_node_id = common_nodes[0]

final_prompt = generate_tans_prompt(
    G,
    sample_node_id,
    node_properties_df,
    classes_cora
)
print("\n--- Final Generated TANS Prompt Example ---")
print(final_prompt)




--- Final Generated TANS Prompt Example ---
Given a node from a citation network graph, where the node type is paper.
The original node description is: "w2 w42 w188 w213 w358 w405 w465 w506 w508 w582 w636 w875 w989 w1072 w1231 w1232 w1259 w1264 w1275 w1394".

The following are the textual information of 5 connected nodes. The descriptions are:
20526: w100 w241 w331 w335 w549 w582 w633 w649 w830 w875 w1072 w1119 w1132 w1156 w1178 w1193 w1207 w1264 w1275 w1360 w1433...
91975: w158 w212 w238 w357 w447 w521 w595 w605 w624 w649 w656 w724 w830 w875 w940 w1072 w1264 w1275 w1309 w1360 w1424...
1114125: w94 w100 w335 w402 w582 w605 w774 w981 w1156 w1178 w1264 w1293 w1307 w1315 w1321 w1382...
39403: w127 w293 w335 w549 w582 w605 w626 w774 w912 w973 w989 w1133 w1156 w1263 w1264 w1293 w1307 w1315 w1321 w1382...

Node Properties:
- Degree Centrality value: 0.0015, ranked as 67.06% among all nodes.
- Closeness Centrality value: 0.1304.
- Betweenness Centrality value: 0.0007.

Output the potential c

In [ ]:
import os
from google import genai
from google.genai.errors import APIError

# Set the Gemini API key as an environment variable. 
# NOTE: The actual key must be replaced for the code to run.
os.environ['GEMINI_API_KEY'] = 'YOUR_API_KEY_HERE'
client = genai.Client()

# Function to query the Gemini model with the TANS prompt and parse the result.
def query_llm_and_generate_description_gemini(prompt, class_list):
    try:
        # Re-instantiate the client (redundant if using global 'client', but often done in functions)
        client = genai.Client()

        # Call the Gemini API to generate content based on the prompt.
        response = client.models.generate_content(
            model='gemini-2.5-flash', # Use the specified model
            contents=prompt           # The TANS prompt containing text and structural data
        )

        llm_explanation = response.text
        predicted_class = None
        
        # Simple heuristic to extract the predicted class: 
        # Check if any class name from the provided list appears in the LLM's explanation (case-insensitive).
        for cls in class_list:
            if cls.lower() in llm_explanation.lower():
                predicted_class = cls
                break

        return predicted_class, llm_explanation

    # Error handling for API-related issues
    except APIError as e:
        print(f"Gemini API Error: {e}")
        return "Error: Could not generate description due to API error.", ""
    # Generic error handling
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "Error: An unexpected error occurred.", ""

# Define the set of possible classes for the Cora dataset.
classes_cora = [
    "Neural Networks", "Probabilistic Methods", "Genetic Algorithms",
    "Theory", "Case Based", "Reinforcement Learning", "Rule Learning"
]

# Run the LLM query using the previously generated 'final_prompt' (assumed to be available).
# This executes the node classification using the LLM.
predicted_class, llm_generated_text = query_llm_and_generate_description_gemini(final_prompt, classes_cora)

# Output the final results: the extracted predicted class and the full LLM-generated reasoning.
print(f"Predicted Class: {predicted_class}")
print(f"Gemini-Generated TANS Description:\n{llm_generated_text}")

Predicted Class: Neural Networks
Gemini-Generated TANS Description:
The node and its connected papers exhibit a strong overlap in their textual descriptions, with several anonymous keywords appearing frequently across multiple nodes. Most notably, `w1264` is present in all five papers, and `w582` appears in four. Keywords `w875`, `w1072`, and `w1275` are also shared extensively.

This high co-occurrence of specific, technical terms suggests a focused research domain with a distinct vocabulary. The node's moderate Degree Centrality (67.06%) indicates it's well-connected within its community, while its low Betweenness Centrality suggests it's not bridging disparate areas. This profile aligns with a paper deeply embedded in a specialized field.

Among the given options, **Neural Networks** are characterized by highly specific architectures, algorithms, and components, which often leads to a concentrated and consistently shared technical vocabulary among related papers. This pattern of foc

In [ ]:
import torch
from torch_geometric.datasets import Planetoid
import networkx as nx
from torch_geometric.utils import to_networkx
import pandas as pd
import numpy as np

# Load the Pubmed dataset
print("--- Loading Pubmed Dataset ---")
dataset = Planetoid(root='./data/Pubmed', name='Pubmed')
data = dataset[0]
print(f"Nodes: {data.num_nodes}, Original Features: {data.num_node_features}, Classes: {dataset.num_classes}")

# Convert to NetworkX
G = to_networkx(data, to_undirected=True)

degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

node_properties_df = pd.DataFrame({
    'node_id': list(G.nodes()),
    'degree_centrality': [degree_centrality[n] for n in G.nodes()],
    'closeness_centrality': [closeness_centrality[n] for n in G.nodes()],
    'betweenness_centrality': [betweenness_centrality[n] for n in G.nodes()]
}).set_index('node_id')

for col in node_properties_df.columns:
    node_properties_df[f'{col}_rank'] = node_properties_df[col].rank(pct=True) * 100


classes_pubmed = ["Experimental Diabetes", "Diabetes Mellitus", "Type 1 Diabetes"] 
sample_node_id = list(G.nodes())[500]
sample_text = "A paper discussing a novel finding related to insulin resistance in mice."

print("Pubmed setup complete.")

--- Loading Pubmed Dataset ---
Nodes: 19717, Original Features: 500, Classes: 3
Pubmed setup complete.


In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

# 'G' and 'data' (Pubmed PyG data object) are assumed to be loaded from previous cells.
# 'all_node_ids' is a list of all node IDs in the Pubmed graph.
all_node_ids = list(G.nodes())


# Function to generate simulated, category-specific text content for a Pubmed node.
# It uses the node's true class label (data.y) to embed a category name into the text.
def generate_placeholder_text(node_id):
    # Get the integer index of the node within the PyG data structure.
    node_index = list(G.nodes()).index(node_id)
    # Retrieve the true numerical class label (0, 1, or 2).
    true_label = data.y[node_index].item()

    # Map the numerical label to the class name string (assumes 'classes_pubmed' is defined).
    class_name = classes_pubmed[true_label]

    # Construct the simulated text string, mentioning the category for demonstration.
    text = (
        f"Research paper ID {node_id}. This article, categorized as '{class_name}', "
        f"investigates the efficacy of novel genetic biomarkers in predicting "
        f"long-term outcomes for patients with {class_name}. "
        f"The findings suggest a strong correlation between the expression of "
        f"the GNG-23 receptor and improved therapeutic response."
    )
    return text

# Create a DataFrame mapping every node ID to its generated placeholder text.
pubmed_texts_df = pd.DataFrame({
    'id': all_node_ids,
    'text': [generate_placeholder_text(n) for n in all_node_ids]
}).set_index('id') # Set the node ID as the index for fast lookup.

print("Simulated Pubmed Text Content (pubmed_texts_df) created.")

# Utility function to retrieve the simulated text content for a specific node ID.
def get_pubmed_text(node_id):
    try:
        return pubmed_texts_df.loc[node_id, 'text']
    except KeyError:
        return f"Text content not found for node ID {node_id}."


# Function to retrieve the simulated text content for a sample of the node's neighbors.
def get_neighbor_texts_pubmed(graph, node_id, num_neighbors=5):
    
    # Check for node existence.
    if node_id not in graph:
        return "Node not found in graph."

    # Get neighbors.
    neighbors = list(graph.neighbors(node_id))

    # Handle case with no neighbors.
    if len(neighbors) == 0:
        return "No connected nodes found."

    # Randomly select a subset of neighbors.
    selected = np.random.choice(
        neighbors,
        min(num_neighbors, len(neighbors)),
        replace=False
    )

    # Format the neighbor output, retrieving and truncating the simulated text.
    return "\n".join(
        [
            f"Node {n}: {get_pubmed_text(n)[:150]}..." # Truncate text for prompt brevity
            for n in selected
        ]
    )

# --- Demonstration and Testing ---

# 'sample_node_id' is assumed to be defined in a previous cell (e.g., node ID 500).
test_node = sample_node_id

print("\n" + "="*50)
print(f"--- TEST RESULTS FOR PUBMED (Node ID: {test_node}) ---")
print("="*50)

# Print the simulated text for the target node.
print("--- Original Text ---")
print(get_pubmed_text(test_node))

# Print the simulated text for a sample of the target node's neighbors.
print("\n--- Neighbor Texts ---")
print(get_neighbor_texts_pubmed(G, test_node, num_neighbors=5))

Simulated Pubmed Text Content (pubmed_texts_df) created.

--- TEST RESULTS FOR PUBMED (Node ID: 500) ---
--- Original Text ---
Research paper ID 500. This article, categorized as 'Type 1 Diabetes', investigates the efficacy of novel genetic biomarkers in predicting long-term outcomes for patients with Type 1 Diabetes. The findings suggest a strong correlation between the expression of the GNG-23 receptor and improved therapeutic response.

--- Neighbor Texts ---
Node 3831: Research paper ID 3831. This article, categorized as 'Type 1 Diabetes', investigates the efficacy of novel genetic biomarkers in predicting long-term ...
Node 17301: Research paper ID 17301. This article, categorized as 'Type 1 Diabetes', investigates the efficacy of novel genetic biomarkers in predicting long-term...
Node 14008: Research paper ID 14008. This article, categorized as 'Type 1 Diabetes', investigates the efficacy of novel genetic biomarkers in predicting long-term...
Node 18728: Research paper ID 18728.

In [ ]:
import numpy as np
import pandas as pd



print("Finding Most Influential Node")

influential_node_info = node_properties_df.sort_values(
    'betweenness_centrality',
    ascending=False
).iloc[0]

most_central_node = influential_node_info.name

print(f"The node with the highest Betweenness Centrality is: {most_central_node}")
print(f"Betweenness Centrality Score: {influential_node_info['betweenness_centrality']:.4f}")



print("\n" + "="*50)
print(f"TEST RESULTS FOR PUBMED (Node ID: {most_central_node} - Highly Central)")
print("="*50)

print("Central Article Text")
print(get_pubmed_text(most_central_node))

print("\n Neighbor Texts (Citing/Cited Articles) ")
print(get_neighbor_texts_pubmed(G, most_central_node, num_neighbors=5))

--- Finding Most Influential Node ---
The node with the highest Betweenness Centrality is: 11024
Betweenness Centrality Score: 0.1429

TEST RESULTS FOR PUBMED (Node ID: 11024 - Highly Central)
Central Article Text
Research paper ID 11024. This article, categorized as 'Type 1 Diabetes', investigates the efficacy of novel genetic biomarkers in predicting long-term outcomes for patients with Type 1 Diabetes. The findings suggest a strong correlation between the expression of the GNG-23 receptor and improved therapeutic response.

 Neighbor Texts (Citing/Cited Articles) 
Node 574: Research paper ID 574. This article, categorized as 'Type 1 Diabetes', investigates the efficacy of novel genetic biomarkers in predicting long-term o...
Node 1416: Research paper ID 1416. This article, categorized as 'Diabetes Mellitus', investigates the efficacy of novel genetic biomarkers in predicting long-ter...
Node 9723: Research paper ID 9723. This article, categorized as 'Type 1 Diabetes', investigates t

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)  
        x = F.dropout(x, p=0.5, training=self.training) 

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
dataset = Planetoid(root='./data/Pubmed', name='Pubmed')
data = dataset[0]

INPUT_DIM = data.num_node_features 
HIDDEN_DIM = 16                    
OUTPUT_DIM = dataset.num_classes   

model = GCN(in_channels=INPUT_DIM, hidden_channels=HIDDEN_DIM, out_channels=OUTPUT_DIM)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss() 

print(f"GCN Model initialized with: Input={INPUT_DIM}, Hidden={HIDDEN_DIM}, Output={OUTPUT_DIM}")

GCN Model initialized with: Input=500, Hidden=16, Output=3


In [ ]:
def train():
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)

    
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    # Backward pass
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
def test():
    model.eval() 
    out = model(data.x, data.edge_index)

    pred = out.argmax(dim=1)

    accuracies = {}

    correct_val = pred[data.val_mask] == data.y[data.val_mask]
    accuracies['val'] = int(correct_val.sum()) / int(data.val_mask.sum())

    correct_test = pred[data.test_mask] == data.y[data.test_mask]
    accuracies['test'] = int(correct_test.sum()) / int(data.test_mask.sum())

    return accuracies

In [124]:
print("\n--- Starting GNN Training ---")
for epoch in range(1, 201):
    loss = train()
    accs = test()

    if epoch % 20 == 0 or epoch == 1:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Val Acc: {accs["val"]:.4f}, Test Acc: {accs["test"]:.4f}')

print("--- Training Complete ---")


--- Starting GNN Training ---
Epoch: 001, Loss: 1.1000, Val Acc: 0.4880, Test Acc: 0.4530
Epoch: 020, Loss: 0.7029, Val Acc: 0.7480, Test Acc: 0.7380
Epoch: 040, Loss: 0.3602, Val Acc: 0.7680, Test Acc: 0.7580
Epoch: 060, Loss: 0.2166, Val Acc: 0.7820, Test Acc: 0.7730
Epoch: 080, Loss: 0.1287, Val Acc: 0.7760, Test Acc: 0.7770
Epoch: 100, Loss: 0.1235, Val Acc: 0.7780, Test Acc: 0.7780
Epoch: 120, Loss: 0.0989, Val Acc: 0.7760, Test Acc: 0.7740
Epoch: 140, Loss: 0.1132, Val Acc: 0.7760, Test Acc: 0.7790
Epoch: 160, Loss: 0.1023, Val Acc: 0.7840, Test Acc: 0.7820
Epoch: 180, Loss: 0.0943, Val Acc: 0.7820, Test Acc: 0.7810
Epoch: 200, Loss: 0.0745, Val Acc: 0.7800, Test Acc: 0.7850
--- Training Complete ---


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = {
    'Epoch': [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
    'Loss': [1.1000, 0.7029, 0.3602, 0.2166, 0.1287, 0.1235, 0.0989, 0.1132, 0.1023, 0.0943, 0.0745],
    'Val Acc': [0.4880, 0.7480, 0.7680, 0.7820, 0.7760,  0.7780, 0.7760, 0.7760, 0.7840, 0.7820, 0.7800],
    'Test Acc': [0.4530, 0.7380, 0.7580, 0.7730, 0.7770, 0.7780, 0.7740, 0.7790, 0.7820, 0.7810, 0.7850]
}
df = pd.DataFrame(data)

fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Training Loss (NLLLoss)', color=color)
ax1.plot(df['Epoch'], df['Loss'], color=color, linestyle='-', marker='o', label='Loss')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, linestyle='--', alpha=0.7)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Accuracy', color=color)
l2, = ax2.plot(df['Epoch'], df['Val Acc'], color='tab:green', linestyle='--', marker='x', label='Validation Accuracy')
l3, = ax2.plot(df['Epoch'], df['Test Acc'], color=color, linestyle='-', marker='s', label='Test Accuracy')
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0.5, 1.0) 

fig.suptitle('GCN Training Performance on Pubmed Dataset', fontsize=16)
lines, labels = ax1.get_legend_handles_labels()
lines.extend([l2, l3])
labels.extend([l2.get_label(), l3.get_label()])
ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.08), ncol=3, frameon=False)

plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
plt.savefig('gnn_training_performance.png')
plt.close()

print("gnn_training_performance.png generated.")

gnn_training_performance.png generated.


In [ ]:
cora_dataset = Planetoid(root='./data/Cora', name='Cora')
data = cora_dataset[0]

INPUT_DIM = data.num_node_features 
HIDDEN_DIM = 16                    
OUTPUT_DIM = cora_dataset.num_classes  

model = GCN(in_channels=INPUT_DIM, hidden_channels=HIDDEN_DIM, out_channels=OUTPUT_DIM)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.NLLLoss() 

print(f"GCN Model initialized with: Input={INPUT_DIM}, Hidden={HIDDEN_DIM}, Output={OUTPUT_DIM}")

GCN Model initialized with: Input=1433, Hidden=16, Output=7


In [ ]:
def train():
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)

    
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    # Backward pass
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
def test():
    model.eval() 
    out = model(data.x, data.edge_index)

    pred = out.argmax(dim=1)

    accuracies = {}

    correct_val = pred[data.val_mask] == data.y[data.val_mask]
    accuracies['val'] = int(correct_val.sum()) / int(data.val_mask.sum())

    correct_test = pred[data.test_mask] == data.y[data.test_mask]
    accuracies['test'] = int(correct_test.sum()) / int(data.test_mask.sum())

    return accuracies

In [140]:
print("\n Starting GNN Training For Cora")
for epoch in range(1, 201):
    loss = train()
    accs = test()

    if epoch % 20 == 0 or epoch == 1:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Val Acc: {accs["val"]:.4f}, Test Acc: {accs["test"]:.4f}')

print("--- Training Complete ---")


 Starting GNN Training For Cora
Epoch: 001, Loss: 1.9527, Val Acc: 0.4420, Test Acc: 0.4500
Epoch: 020, Loss: 0.2934, Val Acc: 0.7740, Test Acc: 0.8040
Epoch: 040, Loss: 0.0805, Val Acc: 0.7700, Test Acc: 0.7810
Epoch: 060, Loss: 0.0450, Val Acc: 0.7740, Test Acc: 0.8060
Epoch: 080, Loss: 0.0260, Val Acc: 0.7800, Test Acc: 0.8090
Epoch: 100, Loss: 0.0389, Val Acc: 0.7800, Test Acc: 0.8060
Epoch: 120, Loss: 0.0342, Val Acc: 0.7620, Test Acc: 0.8000
Epoch: 140, Loss: 0.0371, Val Acc: 0.7720, Test Acc: 0.8000
Epoch: 160, Loss: 0.0376, Val Acc: 0.7700, Test Acc: 0.8040
Epoch: 180, Loss: 0.0214, Val Acc: 0.7700, Test Acc: 0.8110
Epoch: 200, Loss: 0.0365, Val Acc: 0.7660, Test Acc: 0.8070
--- Training Complete ---


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = {
    'Epoch': [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
    'Loss': [1.9527, 0.2934, 0.0805, 0.0450, 0.0260, 0.0389, 0.0342, 0.0371, 0.0376, 0.0214, 0.0365],
    'Val Acc': [0.4420, 0.7740, 0.7700, 0.7740, 0.7800, 0.7800, 0.7620, 0.7720, 0.7700, 0.7700, 0.7660],
    'Test Acc': [0.4500, 0.8040, 0.7810, 0.8060, 0.8090, 0.8060, 0.8000, 0.8000, 0.8040, 0.8110, 0.8070]
}
cora_df = pd.DataFrame(data)

fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Training Loss (NLLLoss)', color=color)
ax1.plot(cora_df['Epoch'], cora_df['Loss'], color=color, linestyle='-', marker='o', label='Loss')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, linestyle='--', alpha=0.7)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Accuracy', color=color)
l2, = ax2.plot(cora_df['Epoch'], cora_df['Val Acc'], color='tab:green', linestyle='--', marker='x', label='Validation Accuracy')
l3, = ax2.plot(cora_df['Epoch'], cora_df['Test Acc'], color=color, linestyle='-', marker='s', label='Test Accuracy')
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0.5, 1.0) 
fig.suptitle('GCN Training Performance on Pubmed Dataset', fontsize=16)
lines, labels = ax1.get_legend_handles_labels()
lines.extend([l2, l3])
labels.extend([l2.get_label(), l3.get_label()])
ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.08), ncol=3, frameon=False)

plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
plt.savefig('gnn_training_performance_cora.png')
plt.close()

print("gnn_training_performance_cora.png generated.")

gnn_training_performance_cora.png generated.


In [ ]:

G_usa_mock = nx.random_geometric_graph(n=1190, radius=0.1)

degree_centrality_usa = nx.degree_centrality(G_usa_mock)

classes_usa = ["High Activity", "Moderate Activity", "Moderately Low Activity", "Low Activity"] # 4 classes
sample_node_id_usa = list(G_usa_mock.nodes())[50]
sample_text_usa = ""


In [ ]:

def get_original_text_text_free(node_id):
    return ""

def get_neighbor_texts_text_free(graph, node_id, num_neighbors=5):
    return "No textual descriptions available for connected nodes."



high_activity_props = node_properties_df.loc[[sample_node_id_usa]].copy()

high_activity_props.loc[sample_node_id_usa, "degree_centrality"] = 0.1749
high_activity_props.loc[sample_node_id_usa, "degree_centrality_rank"] = 99.58  

final_prompt_usa = generate_tans_prompt(
    G_usa_mock,
    sample_node_id_usa,
    high_activity_props,
    classes_usa
)

print("\n--- Final Generated TANS Prompt Example (USA - Text-Free) ---")
print(final_prompt_usa)



--- Final Generated TANS Prompt Example (USA - Text-Free) ---
Given a node from a citation network graph, where the node type is paper.
The original node description is: "No text found for this node.".

The following are the textual information of 5 connected nodes. The descriptions are:
744: No text found for this node....
518: No text found for this node....
952: No text found for this node....
532: No text found for this node....
226: No text found for this node....

Node Properties:
- Degree Centrality value: 0.1749, ranked as 99.58% among all nodes.
- Closeness Centrality value: 0.1458.
- Betweenness Centrality value: 0.0000.

Output the potential class of the node among the following classes: ['High Activity', 'Moderate Activity', 'Moderately Low Activity', 'Low Activity']. 
Provide reasons for your assessment. Your answer should be less than 200 words.
